In [1]:
### Carga de paquetes y librerias
#!pip install keras-tuner
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf #Framework para deep learning
from tensorflow import keras #API que contiene la mayoría de funciones para las RN
from keras_tuner.tuners import RandomSearch

2024-05-20 19:25:01.206437: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
### Carga de datos fasion_mnist
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

### Estandarización de los datos
x_train2 = x_train/255 # Valores entre 0 y 1
x_test2 = x_test/255

# Definicion del modelo para Tunear

In [9]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))

    ## Definición de la primera Capa Oculta con ajute de hiperparametros
    ## Elegir numero optimo de neuronas
    hp_unit_1 = hp.Int('unit_1', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_unit_1, activation='relu'))

    ## Definición de capa de dropout
    ## Elegir el dropout rate optimo - Regularizacion
    hp_dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.3, step=0.1)
    model.add(keras.layers.Dropout(hp_dropout_rate))

    ## Definición de segunda capa oculta con ajuste de hiperparametros
    ## Elegir funcion de activacion optima
    hp_activation_2 = hp.Choice('activation', values=['relu', 'tanh'])
    hp_unit_2 = hp.Int('unit_2', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_unit_2, activation=hp_activation_2))

    ## Definicion de la capa de salida
    model.add(keras.layers.Dense(10, activation='softmax'))

    ## Definicion de la tasa de aprendizaje (learning_rate) del optimizador
    ## Elegir un valor optimo entre 0.1, 0.01, 0.001, 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[0.1, 0.01, 0.001, 0.0001])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [10]:
### Definición del Tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2,
    directory='results_tuner',
    project_name='Demo'
)

In [12]:
## Ejecutar tuning de hiperparametros
tuner.search(x_train2, y_train, epochs=5, validation_data=(x_test2, y_test))

Trial 5 Complete [00h 01m 36s]
val_accuracy: 0.14910000190138817

Best val_accuracy So Far: 0.8814999759197235
Total elapsed time: 00h 07m 13s


In [15]:
tuner.get_best_hyperparameters()[0].get('activation')

'relu'

In [17]:
### Mostrar el mejor modelo
for h_param in [f"unit_{i}" for i in range(1,3)] + ['activation'] + ['learning_rate']:
    print(h_param, tuner.get_best_hyperparameters()[0].get(h_param))
    


unit_1 320
unit_2 448
activation relu
learning_rate 0.001


In [ ]:
### Almacenar el mejor modelo
best_model = tuner.get_best_models()[0]
### Definir la arquitectura del modelo según hiperparámetros optimos
best_model.build(x_train2.shape)
### Resumen de la arquitectura
best_model.summary()